In [5]:
import torch
model = torch.hub.load('../yolov7', 'custom', 'best.pt', source='local')


                 from  n    params  module                                  arguments                     
  0                -1  1      1160  models.common.Conv                      [3, 40, 3, 1]                 
  1                -1  1     28960  models.common.Conv                      [40, 80, 3, 2]                
  2                -1  1     57760  models.common.Conv                      [80, 80, 3, 1]                
  3                -1  1    115520  models.common.Conv                      [80, 160, 3, 2]               
  4                -1  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  5                -2  1     10368  models.common.Conv                      [160, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

 76                28  1    102720  models.common.Conv                      [640, 160, 1, 1]              
 77          [-1, -2]  1         0  models.common.Concat                    [1]                           
 78                -1  1     41216  models.common.Conv                      [320, 128, 1, 1]              
 79                -2  1     41216  models.common.Conv                      [320, 128, 1, 1]              
 80                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 81                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 82                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 83                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 84                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 85                -1  1    147712  m

Adding autoShape... 


In [6]:
import cv2
import numpy as np


class kalman_filter:
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def predict(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y

In [ ]:
import cv2
import numpy as np
kf = kalman_filter()
# Initialize the Kalman filter
kalman = cv2.KalmanFilter(4, 2)  # 4 states, 2 measurements

# Set the transition matrix
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                   [0, 1, 0, 1],
                                   [0, 0, 1, 0],
                                   [0, 0, 0, 1]], np.float32)

# Set the initial state
kalman.statePre = np.array([[0],
                            [0],
                            [0],
                            [0]], np.float32)

# Set the measurement matrix
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)

# Set the measurement noise covariance matrix
kalman.measurementNoiseCov = np.array([[1, 0],
                                       [0, 1]], np.float32) * 1e-2

# Set the process noise covariance matrix
kalman.processNoiseCov = np.array([[1, 0, 1, 0],
                                   [0, 1, 0, 1],
                                   [1, 0, 1, 0],
                                   [0, 1, 0, 1]], np.float32) * 1e-5

# Load the video
cap = cv2.VideoCapture('1.mp4')
X = []
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

     # Perform object detection on the frame
    detections = model(frame)
    results = detections.pandas().xyxy[0].to_dict(orient="records")

    # Process the detections
    results = detections.pandas().xyxy[0].to_dict(orient="records")
    for result in results:
        # Process each detected object
        print(result)

    # Display the frame with bounding boxes

        
    for result in results:
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])
                # Do whatever you want
                x = int((x1+x2)/2)
                y = int((y1+y2)/2)
                X.append((x,y))

                frame1 = cv2.rectangle(frame, (x1, x2), (y1, y2), (0, 255, 0), 1)
                for pt in X:
                    cv2.circle(frame1, pt, 2, (0, 20, 220), -1)
                    predicted = kf.predict(pt[0], pt[1])
                for i in range(10):
                    predicted = kf.predict(predicted[0], predicted[1])
                    cv2.circle(frame1, predicted, 2, (20, 220, 0), 1)

                cv2.rectangle(frame, (x1, y1), (x2, y2),(0,50 ,200 ), 2)
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    # Perform some image processing or object detection to get the measurement
    # In this example, we assume the measurement is the centroid of a detected object
    x, y = 100, 100  # Sample measurement values

    # Apply the Kalman filter
    measurement = np.array([[x], [y]], np.float32)
    kalman.correct(measurement)
    prediction = kalman.predict()

    # Extract the predicted coordinates
    predicted_x, predicted_y = prediction[0][0], prediction[1][0]

    # Draw the predicted coordinates on the frame
    cv2.circle(frame, (int(predicted_x), int(predicted_y)), 1, (0, 255, 0), -1)

    # Display the frames
#     cv2.imshow('Object Detection', frame)
#     cv2.imshow('Object', frame1)
    cv2.imshow('Kalman Filter', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video and close windows
cap.release()
cv2.destroyAllWindows()


{'xmin': 0.640756368637085, 'ymin': 155.68016052246094, 'xmax': 7.708931922912598, 'ymax': 159.0355987548828, 'confidence': 0.3318628966808319, 'class': 2, 'name': 'drone'}
{'xmin': 0.3600308895111084, 'ymin': 155.57186889648438, 'xmax': 7.645978927612305, 'ymax': 159.07736206054688, 'confidence': 0.3080897033214569, 'class': 1, 'name': 'plane'}
{'xmin': 3.6749768257141113, 'ymin': 155.36231994628906, 'xmax': 10.559450149536133, 'ymax': 159.15003967285156, 'confidence': 0.4190479516983032, 'class': 2, 'name': 'drone'}
{'xmin': 6.734984397888184, 'ymin': 155.2578582763672, 'xmax': 13.772688865661621, 'ymax': 159.3692169189453, 'confidence': 0.34235110878944397, 'class': 2, 'name': 'drone'}
{'xmin': 9.673651695251465, 'ymin': 154.78880310058594, 'xmax': 16.598102569580078, 'ymax': 159.3820037841797, 'confidence': 0.31149840354919434, 'class': 0, 'name': 'bird'}
{'xmin': 13.196491241455078, 'ymin': 154.35394287109375, 'xmax': 20.59581756591797, 'ymax': 159.5911865234375, 'confidence': 0.3

{'xmin': 135.85679626464844, 'ymin': 159.1834716796875, 'xmax': 143.85765075683594, 'ymax': 163.85498046875, 'confidence': 0.733346700668335, 'class': 2, 'name': 'drone'}
{'xmin': 139.43182373046875, 'ymin': 160.3186492919922, 'xmax': 145.92074584960938, 'ymax': 163.95069885253906, 'confidence': 0.7227979898452759, 'class': 2, 'name': 'drone'}
{'xmin': 142.26251220703125, 'ymin': 160.75672912597656, 'xmax': 149.58718872070312, 'ymax': 164.48814392089844, 'confidence': 0.6459725499153137, 'class': 2, 'name': 'drone'}
{'xmin': 144.53285217285156, 'ymin': 160.8837127685547, 'xmax': 152.37855529785156, 'ymax': 165.20237731933594, 'confidence': 0.6004915237426758, 'class': 2, 'name': 'drone'}
{'xmin': 148.1355743408203, 'ymin': 161.07742309570312, 'xmax': 154.89976501464844, 'ymax': 165.85031127929688, 'confidence': 0.6334210634231567, 'class': 2, 'name': 'drone'}
{'xmin': 150.73199462890625, 'ymin': 161.45372009277344, 'xmax': 157.8155517578125, 'ymax': 166.33982849121094, 'confidence': 0.

{'xmin': 276.99578857421875, 'ymin': 172.76718139648438, 'xmax': 284.9735107421875, 'ymax': 177.84078979492188, 'confidence': 0.7214257717132568, 'class': 2, 'name': 'drone'}
{'xmin': 279.72332763671875, 'ymin': 172.96519470214844, 'xmax': 288.2578125, 'ymax': 178.0575714111328, 'confidence': 0.8256760239601135, 'class': 2, 'name': 'drone'}
{'xmin': 34.03489685058594, 'ymin': 11.636347770690918, 'xmax': 38.63795471191406, 'ymax': 15.852364540100098, 'confidence': 0.4419110417366028, 'class': 0, 'name': 'bird'}
{'xmin': 34.60377883911133, 'ymin': 12.187376976013184, 'xmax': 39.99229049682617, 'ymax': 16.714035034179688, 'confidence': 0.46018388867378235, 'class': 0, 'name': 'bird'}
{'xmin': 35.33995819091797, 'ymin': 13.070169448852539, 'xmax': 40.35862731933594, 'ymax': 16.826372146606445, 'confidence': 0.3952467441558838, 'class': 0, 'name': 'bird'}
{'xmin': 35.685550689697266, 'ymin': 13.350049018859863, 'xmax': 40.69840621948242, 'ymax': 16.940549850463867, 'confidence': 0.341100037

{'xmin': 67.9811019897461, 'ymin': 29.68379783630371, 'xmax': 73.70381927490234, 'ymax': 34.67112350463867, 'confidence': 0.46958741545677185, 'class': 0, 'name': 'bird'}
{'xmin': 68.91370391845703, 'ymin': 30.433488845825195, 'xmax': 74.01366424560547, 'ymax': 35.01994705200195, 'confidence': 0.4114232361316681, 'class': 0, 'name': 'bird'}
{'xmin': 69.69306945800781, 'ymin': 30.615060806274414, 'xmax': 74.50404357910156, 'ymax': 35.23887634277344, 'confidence': 0.3565766215324402, 'class': 0, 'name': 'bird'}
{'xmin': 70.25554656982422, 'ymin': 30.905319213867188, 'xmax': 75.56597137451172, 'ymax': 35.53105163574219, 'confidence': 0.36366140842437744, 'class': 0, 'name': 'bird'}
{'xmin': 70.58553314208984, 'ymin': 31.539112091064453, 'xmax': 75.56575775146484, 'ymax': 35.83193588256836, 'confidence': 0.3329521119594574, 'class': 0, 'name': 'bird'}
{'xmin': 70.84950256347656, 'ymin': 31.717052459716797, 'xmax': 75.70265197753906, 'ymax': 35.90605545043945, 'confidence': 0.25449621677398

In [ ]:
## random stuff

In [11]:
import cv2

video_path = '1.mp4'
video = cv2.VideoCapture(video_path)

# Process each frame in the video
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Perform object detection on the frame
    detections = model(frame)
    results = detections.pandas().xyxy[0].to_dict(orient="records")

    # Process the detections
    results = detections.pandas().xyxy[0].to_dict(orient="records")
    for result in results:
        # Process each detected object
        print(result)

    # Display the frame with bounding boxes
    cv2.imshow('Object Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video file and close windows
video.release()
cv2.destroyAllWindows()

C:\Users\HP\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


{'xmin': 0.640756368637085, 'ymin': 155.68016052246094, 'xmax': 7.708931922912598, 'ymax': 159.0355987548828, 'confidence': 0.3318628966808319, 'class': 2, 'name': 'drone'}
{'xmin': 0.3600308895111084, 'ymin': 155.57186889648438, 'xmax': 7.645978927612305, 'ymax': 159.07736206054688, 'confidence': 0.3080897033214569, 'class': 1, 'name': 'plane'}
{'xmin': 3.6749768257141113, 'ymin': 155.36231994628906, 'xmax': 10.559450149536133, 'ymax': 159.15003967285156, 'confidence': 0.4190479516983032, 'class': 2, 'name': 'drone'}
{'xmin': 6.734984397888184, 'ymin': 155.2578582763672, 'xmax': 13.772688865661621, 'ymax': 159.3692169189453, 'confidence': 0.34235110878944397, 'class': 2, 'name': 'drone'}
{'xmin': 9.673651695251465, 'ymin': 154.78880310058594, 'xmax': 16.598102569580078, 'ymax': 159.3820037841797, 'confidence': 0.31149840354919434, 'class': 0, 'name': 'bird'}
{'xmin': 13.196491241455078, 'ymin': 154.35394287109375, 'xmax': 20.59581756591797, 'ymax': 159.5911865234375, 'confidence': 0.3

{'xmin': 135.85679626464844, 'ymin': 159.1834716796875, 'xmax': 143.85765075683594, 'ymax': 163.85498046875, 'confidence': 0.733346700668335, 'class': 2, 'name': 'drone'}
{'xmin': 139.43182373046875, 'ymin': 160.3186492919922, 'xmax': 145.92074584960938, 'ymax': 163.95069885253906, 'confidence': 0.7227979898452759, 'class': 2, 'name': 'drone'}
{'xmin': 142.26251220703125, 'ymin': 160.75672912597656, 'xmax': 149.58718872070312, 'ymax': 164.48814392089844, 'confidence': 0.6459725499153137, 'class': 2, 'name': 'drone'}
{'xmin': 144.53285217285156, 'ymin': 160.8837127685547, 'xmax': 152.37855529785156, 'ymax': 165.20237731933594, 'confidence': 0.6004915237426758, 'class': 2, 'name': 'drone'}
{'xmin': 148.1355743408203, 'ymin': 161.07742309570312, 'xmax': 154.89976501464844, 'ymax': 165.85031127929688, 'confidence': 0.6334210634231567, 'class': 2, 'name': 'drone'}
{'xmin': 150.73199462890625, 'ymin': 161.45372009277344, 'xmax': 157.8155517578125, 'ymax': 166.33982849121094, 'confidence': 0.

{'xmin': 276.99578857421875, 'ymin': 172.76718139648438, 'xmax': 284.9735107421875, 'ymax': 177.84078979492188, 'confidence': 0.7214257717132568, 'class': 2, 'name': 'drone'}
{'xmin': 279.72332763671875, 'ymin': 172.96519470214844, 'xmax': 288.2578125, 'ymax': 178.0575714111328, 'confidence': 0.8256760239601135, 'class': 2, 'name': 'drone'}
{'xmin': 34.03489685058594, 'ymin': 11.636347770690918, 'xmax': 38.63795471191406, 'ymax': 15.852364540100098, 'confidence': 0.4419110417366028, 'class': 0, 'name': 'bird'}
{'xmin': 34.60377883911133, 'ymin': 12.187376976013184, 'xmax': 39.99229049682617, 'ymax': 16.714035034179688, 'confidence': 0.46018388867378235, 'class': 0, 'name': 'bird'}
{'xmin': 35.33995819091797, 'ymin': 13.070169448852539, 'xmax': 40.35862731933594, 'ymax': 16.826372146606445, 'confidence': 0.3952467441558838, 'class': 0, 'name': 'bird'}
{'xmin': 35.685550689697266, 'ymin': 13.350049018859863, 'xmax': 40.69840621948242, 'ymax': 16.940549850463867, 'confidence': 0.341100037

{'xmin': 67.9811019897461, 'ymin': 29.68379783630371, 'xmax': 73.70381927490234, 'ymax': 34.67112350463867, 'confidence': 0.46958741545677185, 'class': 0, 'name': 'bird'}
{'xmin': 68.91370391845703, 'ymin': 30.433488845825195, 'xmax': 74.01366424560547, 'ymax': 35.01994705200195, 'confidence': 0.4114232361316681, 'class': 0, 'name': 'bird'}
{'xmin': 69.69306945800781, 'ymin': 30.615060806274414, 'xmax': 74.50404357910156, 'ymax': 35.23887634277344, 'confidence': 0.3565766215324402, 'class': 0, 'name': 'bird'}
{'xmin': 70.25554656982422, 'ymin': 30.905319213867188, 'xmax': 75.56597137451172, 'ymax': 35.53105163574219, 'confidence': 0.36366140842437744, 'class': 0, 'name': 'bird'}
{'xmin': 70.58553314208984, 'ymin': 31.539112091064453, 'xmax': 75.56575775146484, 'ymax': 35.83193588256836, 'confidence': 0.3329521119594574, 'class': 0, 'name': 'bird'}
{'xmin': 70.84950256347656, 'ymin': 31.717052459716797, 'xmax': 75.70265197753906, 'ymax': 35.90605545043945, 'confidence': 0.25449621677398

{'xmin': 97.43914031982422, 'ymin': 53.55449676513672, 'xmax': 102.42279815673828, 'ymax': 57.127113342285156, 'confidence': 0.39675721526145935, 'class': 0, 'name': 'bird'}
{'xmin': 19.678476333618164, 'ymin': 28.911075592041016, 'xmax': 25.22345542907715, 'ymax': 32.95711898803711, 'confidence': 0.30512699484825134, 'class': 0, 'name': 'bird'}
{'xmin': 22.07943344116211, 'ymin': 28.900798797607422, 'xmax': 27.522743225097656, 'ymax': 33.66959762573242, 'confidence': 0.41693830490112305, 'class': 0, 'name': 'bird'}
{'xmin': 97.70645141601562, 'ymin': 53.78977966308594, 'xmax': 102.94334411621094, 'ymax': 57.70391845703125, 'confidence': 0.37473973631858826, 'class': 0, 'name': 'bird'}
{'xmin': 23.97423553466797, 'ymin': 29.40700340270996, 'xmax': 29.24566650390625, 'ymax': 34.00072479248047, 'confidence': 0.4853938817977905, 'class': 0, 'name': 'bird'}
{'xmin': 97.97541046142578, 'ymin': 54.05073547363281, 'xmax': 103.40068817138672, 'ymax': 58.159202575683594, 'confidence': 0.4179680

{'xmin': 76.68852996826172, 'ymin': 40.167457580566406, 'xmax': 81.10907745361328, 'ymax': 44.083160400390625, 'confidence': 0.44045835733413696, 'class': 0, 'name': 'bird'}
{'xmin': 111.2516860961914, 'ymin': 62.30878448486328, 'xmax': 117.88140106201172, 'ymax': 67.4435806274414, 'confidence': 0.2976435124874115, 'class': 0, 'name': 'bird'}
{'xmin': 78.25487518310547, 'ymin': 40.47718811035156, 'xmax': 82.88689422607422, 'ymax': 44.43583679199219, 'confidence': 0.3842891752719879, 'class': 0, 'name': 'bird'}
{'xmin': 80.1556396484375, 'ymin': 41.02901077270508, 'xmax': 85.21823120117188, 'ymax': 45.00581741333008, 'confidence': 0.435351699590683, 'class': 0, 'name': 'bird'}
{'xmin': 115.49060821533203, 'ymin': 65.42599487304688, 'xmax': 118.66339874267578, 'ymax': 68.34878540039062, 'confidence': 0.2543487250804901, 'class': 0, 'name': 'bird'}
{'xmin': 82.16331481933594, 'ymin': 41.40775680541992, 'xmax': 87.10707092285156, 'ymax': 45.22036361694336, 'confidence': 0.4189417362213135,

{'xmin': 132.8060760498047, 'ymin': 57.08798599243164, 'xmax': 138.30735778808594, 'ymax': 61.36796188354492, 'confidence': 0.36723148822784424, 'class': 2, 'name': 'drone'}
{'xmin': 132.858642578125, 'ymin': 57.08972930908203, 'xmax': 138.33798217773438, 'ymax': 61.309471130371094, 'confidence': 0.28694117069244385, 'class': 0, 'name': 'bird'}
{'xmin': 134.6402130126953, 'ymin': 57.73334503173828, 'xmax': 139.75721740722656, 'ymax': 62.06608581542969, 'confidence': 0.44065341353416443, 'class': 0, 'name': 'bird'}
{'xmin': 131.3025665283203, 'ymin': 75.7366714477539, 'xmax': 134.27845764160156, 'ymax': 78.5636978149414, 'confidence': 0.2518141269683838, 'class': 0, 'name': 'bird'}
{'xmin': 136.3211212158203, 'ymin': 58.11829376220703, 'xmax': 141.0070343017578, 'ymax': 62.28954315185547, 'confidence': 0.4656822979450226, 'class': 0, 'name': 'bird'}
{'xmin': 138.2388153076172, 'ymin': 58.84599304199219, 'xmax': 142.50328063964844, 'ymax': 62.79283905029297, 'confidence': 0.3549517989158

{'xmin': 204.2782440185547, 'ymin': 84.7635726928711, 'xmax': 208.32969665527344, 'ymax': 88.1335220336914, 'confidence': 0.3501771092414856, 'class': 0, 'name': 'bird'}
{'xmin': 205.450439453125, 'ymin': 84.65300750732422, 'xmax': 210.50125122070312, 'ymax': 88.32979583740234, 'confidence': 0.3665757477283478, 'class': 2, 'name': 'drone'}
{'xmin': 205.67837524414062, 'ymin': 84.41064453125, 'xmax': 210.56106567382812, 'ymax': 88.2181396484375, 'confidence': 0.3544798493385315, 'class': 0, 'name': 'bird'}
{'xmin': 207.78680419921875, 'ymin': 84.95577239990234, 'xmax': 212.11862182617188, 'ymax': 88.4964828491211, 'confidence': 0.45067209005355835, 'class': 0, 'name': 'bird'}
{'xmin': 208.4551544189453, 'ymin': 85.2015380859375, 'xmax': 213.32203674316406, 'ymax': 88.74996948242188, 'confidence': 0.4162023663520813, 'class': 2, 'name': 'drone'}
{'xmin': 208.503173828125, 'ymin': 85.07740783691406, 'xmax': 213.14193725585938, 'ymax': 88.70726013183594, 'confidence': 0.27867844700813293, 

In [1]:
import subprocess as sr

sr.run(['python','detect.py', '--source','1.mp4','--weights','best.pt','--view-img'])

CompletedProcess(args=['python', 'detect.py', '--source', '1.mp4', '--weights', 'best.pt', '--view-img'], returncode=0)